# Simplifed ResNet implementation

## Loading CIFAR10

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras

In [ ]:

from tensorflow.keras.datasets import cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 4s 0us/step


In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)
(50000, 1)
(10000, 1)


In [ ]:
# normalization
X_train, X_test = X_train/255.0, X_test/255.0

## Construct Resnet

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.layers import Add, AveragePooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

In [ ]:
inputs = Input(shape=(32,32,3))
x = Conv2D(32, kernel_size=(3,3), strides=(1,1),
           padding='same', use_bias=False)(inputs)
x = BatchNormalization()(x)
x = ReLU()(x)

x = Conv2D(32, kernel_size=(3,3), strides=(1,1),
           padding='same', use_bias=False)(x)
x = BatchNormalization()(x)
x = ReLU()(x)

x = MaxPool2D(pool_size=(2,2), strides=(2,2),
              padding='valid')(x)
x = Conv2D(64, kernel_size=(3,3), strides=(1,1),
           padding='same', use_bias=False)(x)
x = BatchNormalization()(x)
x = ReLU()(x)

# 1st skip connection
skip = x
x = Conv2D(64, kernel_size=(3,3), strides=(1,1),
           padding='same', use_bias=False)(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Conv2D(64, kernel_size=(3,3), strides=(1,1),
           padding='same', use_bias=False)(x)
x = BatchNormalization()(x)

x = Add()([x, skip])
x = ReLU()(x)

x = MaxPool2D(pool_size=(2,2), strides=(2,2),
              padding='valid')(x)
x = Conv2D(128, kernel_size=(3,3), strides=(1,1),
           padding='same', use_bias=False)(x)
x = BatchNormalization()(x)
x = ReLU()(x)

# 2nd skip connection
skip = x
x = Conv2D(128, kernel_size=(3,3), strides=(1,1),
           padding='same', use_bias=False)(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Conv2D(128, kernel_size=(3,3), strides=(1,1),
           padding='same', use_bias=False)(x)
x = BatchNormalization()(x)

x = Add()([x, skip])
x = ReLU()(x)

# Average Pooling
x = AveragePooling2D(pool_size=(8,8))(x)
x = Flatten()(x)
x = ReLU()(x)
outputs = Dense(10, activation='softmax')(x)

Model_resnet = Model(inputs = inputs, outputs = outputs)

Model_resnet.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 32, 32, 32)           864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 32, 32, 32)           128       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 re_lu (ReLU)                (None, 32, 32, 32)           0         ['batch_normalization[0][0

## Compile

In [ ]:
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate = 0.001,
          beta_1 = 0.9,
          beta_2 = 0.999)

Model_resnet.compile(optimizer = opt,
                    loss='sparse_categorical_crossentropy',
                    metrics=['acc'])

In [ ]:
# training
Model_resnet.fit(X_train,y_train,epochs=20)

Epoch 1/20
1563/1563 [==============================] - 20s 7ms/step - loss: 1.1865 - acc: 0.5755
Epoch 2/20
1563/1563 [==============================] - 11s 7ms/step - loss: 0.7655 - acc: 0.7348
Epoch 3/20
1563/1563 [==============================] - 11s 7ms/step - loss: 0.6135 - acc: 0.7881
Epoch 4/20
1563/1563 [==============================] - 11s 7ms/step - loss: 0.5183 - acc: 0.8217
Epoch 5/20
1563/1563 [==============================] - 11s 7ms/step - loss: 0.4413 - acc: 0.8480
Epoch 6/20
1563/1563 [==============================] - 11s 7ms/step - loss: 0.3780 - acc: 0.8703
Epoch 7/20
1563/1563 [==============================] - 12s 7ms/step - loss: 0.3255 - acc: 0.8867
Epoch 8/20
1563/1563 [==============================] - 11s 7ms/step - loss: 0.2763 - acc: 0.9050
Epoch 9/20
1563/1563 [==============================] - 11s 7ms/step - loss: 0.2351 - acc: 0.9167
Epoch 10/20
1563/1563 [==============================] - 11s 7ms/step - loss: 0.1979 - acc: 0.9298
Epoch 11/20
1563/15

In [ ]:
# Evaluation
test_performance = Model_resnet.evaluate(X_test,y_test)
print(test_performance)

313/313 [==============================] - 2s 4ms/step - loss: 0.7864 - acc: 0.8240
[0.7863873243331909, 0.8240000009536743]
